In [34]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import check_classification_targets

class HQC_original(BaseEstimator, ClassifierMixin):
    """The Helstrom Quantum Centroid (HQC) classifier is a quantum-inspired supervised classification 
    approach for data with binary classes (ie. data with 2 classes only). By quantum-inspired, we 
    mean a classification process which employs and exploits Quantum Theory. It is inspired by the 
    quantum Helstrom observable which acts on the distinguishability between quantum patterns rather 
    than classical patterns of a dataset. The classical dataset is encoded into quantum densities 
    using the inverse of the standard stereographic projection encoding method. There is an option
    to rescale the dataset and to choose the number of copies to take for the quantum densities.
                         
    Parameters
    ----------
    rescale : int, default = 1
        The dataset rescaling factor. A parameter used for rescaling the dataset. 
    n_copies : int, default = 1
        The number of copies to take for each quantum density. This is equivalent to taking the 
        n-fold Kronecker tensor product for each quantum density.       

    Attributes
    ----------
    classes_ : ndarray, shape (2,)
        Sorted binary classes.
    centroid_class_0_ : ndarray, shape (n_features + 1, n_features + 1)
        Quantum Centroid for class with index 0.
    centroid_class_1_ : ndarray, shape (n_features + 1, n_features + 1)
        Quantum Centroid for class with index 1.
    q_Hels_obs_ : ndarray, shape (n_features + 1, n_features + 1)
        Quantum Helstrom observable.
    proj_pos_ : ndarray, shape (n_features + 1, n_features + 1)
        Sum of the projectors of the Quantum Helstrom observable's eigenvectors, which has 
        corresponding positive eigenvalues.
    proj_neg_ : ndarray, shape (n_features + 1, n_features + 1)
        Sum of the projectors of the Quantum Helstrom observable's eigenvectors, which has 
        corresponding negative eigenvalues.
    Hels_bound_ : float
        Helstrom bound is the upper bound of the probability that one can correctly discriminate 
        whether a quantum density is of which of the two binary quantum density pattern.          
    """
    # Added binary_only tag as required by sklearn check_estimator
    def _more_tags(self):
        return {'binary_only': True}
    
    
    def __init__(self, rescale=1, n_copies=1):
        self.rescale = rescale
        self.n_copies = n_copies
        
        
    def fit(self, X, y):
        """Perform HQC classification with the inverse of the standard stereographic projection encoding, 
        with the option to rescale the dataset prior to encoding.
                
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples. An array of int or float.
        y : array-like, shape (n_samples,)
            The training input binary target values. An array of str, int or float.
            
        Returns
        -------
        self : object
            Returns self.
        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Ensure target y is of non-regression type
        # Added as required by sklearn check_estimator
        check_classification_targets(y)
    
        # Store binary classes and encode y into binary class indexes 0 and 1
        self.classes_, y_class_index = np.unique(y, return_inverse=True)
        
        # Cast X to float to ensure all following calculations below are done in float rather than int 
        X = X.astype(float)
        
        # Rescale X
        X = self.rescale*X
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(axis=1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]
        
        # Initialize array X_prime
        X_prime = np.empty((m,n+1))
        # Calculate X'
        for i in range(0,m):
            X_prime[i,:] = (1/(X_sq_sum[i]+1))*(np.concatenate((2*X,(X_sq_sum-1).reshape((-1,1))),axis=1)[i,:])
        
        # Determine rows (samples) in X' belonging to class index 0
        X_prime_class_0 = X_prime[y_class_index==0]
        
        # Determine rows (samples) in X' belonging to class index 1
        X_prime_class_1 = X_prime[y_class_index==1]
        
        # Number of rows (samples) in X'
        M = m
        
        # Number of rows (samples) in X' belonging to class index 0
        M_class_0 = X_prime_class_0.shape[0]
        
        # Number of rows (samples) in X' belonging to class index 1
        M_class_1 = X_prime_class_1.shape[0]
        
        # Initialize array density_class_0
        density_class_0 = np.zeros(((n+1)**self.n_copies,(n+1)**self.n_copies))
        for i in range(0,M_class_0):
            # Encode into quantum densities by using the inverse of the standard stereographic projection 
            # encoding method 
            density_each_row = np.dot(X_prime_class_0[i,:].reshape(-1,1),X_prime_class_0[i,:].reshape(1,-1))
            
            # Calculate n-fold Kronecker tensor product
            if self.n_copies==1:
                density_each_row = density_each_row
            else:
                density_each_row_copy = density_each_row
                for j in range(0,self.n_copies-1):
                    density_each_row = np.kron(density_each_row,density_each_row_copy)
                    
            # Calculate sum of quantum densities belonging to class index 0
            density_class_0 = density_class_0 + density_each_row
            
        # Calculate Quantum Centroid for class index 0
        self.centroid_class_0_ = (1/M_class_0)*density_class_0
        
        # Initialize array density_class_1
        density_class_1 = np.zeros(((n+1)**self.n_copies,(n+1)**self.n_copies))
        for i in range(0,M_class_1):
            # Encode into quantum densities by using the inverse of the standard stereographic projection 
            # encoding method
            density_each_row = np.dot(X_prime_class_1[i,:].reshape(-1,1),X_prime_class_1[i,:].reshape(1,-1))
            
            # Calculate n-fold Kronecker tensor product
            if self.n_copies==1:
                density_each_row = density_each_row
            else:
                density_each_row_copy = density_each_row
                for j in range(0,self.n_copies-1):
                    density_each_row = np.kron(density_each_row,density_each_row_copy)
                    
            # Calculate sum of quantum densities belonging to class index 1        
            density_class_1 = density_class_1 + density_each_row
            
        # Calculate Quantum Centroid for class index 1
        # Added ZeroDivisionError as required by sklearn check_estimator
        try:
            self.centroid_class_1_ = (1/M_class_1)*density_class_1
        except ZeroDivisionError:
            self.centroid_class_1_ = 0

        # Calculate quantum Helstrom observable
        self.q_Hels_obs_ = (M_class_0/M)*self.centroid_class_0_ - (M_class_1/M)*self.centroid_class_1_
        
        # Calculate eigenvalues w and eigenvectors v of the quantum Helstrom observable
        w, v = np.linalg.eig(self.q_Hels_obs_)
        
        # Length of w
        len_w = len(w)
        
        # Initialize arrays self.proj_pos_ and self.proj_neg_
        self.proj_pos_ = np.zeros_like(self.q_Hels_obs_)
        self.proj_neg_ = np.zeros_like(self.q_Hels_obs_)
        # Calculate sum of projectors of eigenvectors with corresponding positive and negative 
        # eigenvalues, respectively
        for i in range(0,len_w):
            if w[i] > 0:
                self.proj_pos_ = self.proj_pos_ + np.real(np.dot(v[:,i].reshape(-1,1),v[:,i].reshape(1,-1)))
            else:
                self.proj_neg_ = self.proj_neg_ + np.real(np.dot(v[:,i].reshape(-1,1),v[:,i].reshape(1,-1)))
    
        # Calculate Helstrom bound
        self.Hels_bound_ = (M_class_0/M)*np.trace(np.dot(self.centroid_class_0_,self.proj_pos_)) \
                           + (M_class_1/M)*np.trace(np.dot(self.centroid_class_1_,self.proj_neg_))
        return self
        
        
    def predict_proba(self, X):
        """Performs HQC classification on X and returns the trace of the dot product of the densities and the 
        sum of the projectors with corresponding positive and negative eigenvalues, respectively.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.       
            
        Returns
        -------
        trace_matrix : array-like, shape (n_samples, 2)
            Column index 0 corresponds to the trace of the dot product of the densities and the sum of 
            projectors with positive eigenvalues. Column index 1 corresponds to the trace of the dot 
            product of the densities and the sum of projectors with negative eigenvalues. An array of float.
        """
        # Check if fit had been called
        check_is_fitted(self, ['proj_pos_', 'proj_neg_'])

        # Input validation
        X = check_array(X)
        
        # Cast X to float to ensure all following calculations below are done in float rather than int 
        X = X.astype(float)        
        
        # Rescale X
        X = self.rescale*X        
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(axis=1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]
        
        # Initialize array X_prime
        X_prime = np.empty((m,n+1))
        # Calculate X'
        for i in range(0,m):
            X_prime[i,:] = (1/(X_sq_sum[i]+1))*(np.concatenate((2*X,(X_sq_sum-1).reshape((-1,1))),axis=1)[i,:])
            
        # Initialize array trace_matrix (which can contain complex numbers)
        trace_matrix = np.empty((m,2))
        for i in range (0,m):
            # Encode into quantum densities by using the inverse of the standard stereographic projection 
            # encoding method
            density_each_row = np.dot(X_prime[i,:].reshape(-1,1),X_prime[i,:].reshape(1,-1))
            
            # Calculate n-fold Kronecker tensor product
            if self.n_copies==1:
                density_each_row = density_each_row
            else:
                density_each_row_copy = density_each_row
                for j in range(0,self.n_copies-1):
                    density_each_row = np.kron(density_each_row,density_each_row_copy)
                    
            # Calculate trace of the dot product of density of each row and sum of projectors with corresponding 
            # positive and negative eigenvalues, respectively
            trace_matrix[i,0] = np.trace(np.dot(density_each_row,self.proj_pos_))
            trace_matrix[i,1] = np.trace(np.dot(density_each_row,self.proj_neg_))
        return trace_matrix
    
    
    def predict(self, X):
        """Performs HQC classification on X and returns the binary classes.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.
            
        Returns
        -------
        self.classes_[predict_trace_index] : array-like, shape (n_samples,)
            The predicted binary classes. An array of str, int or float.
        """
        # Determine column index with the higher trace value in trace_matrix
        # If both columns have the same trace value, returns column index 0
        predict_trace_index = np.argmax(self.predict_proba(X), axis=1)
        # Returns the predicted binary classes
        return self.classes_[predict_trace_index]

In [35]:
# appendicitis dataset (7 features, 106 rows)
import pandas as pd

df = pd.read_csv('appendicitis.tsv',delimiter='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

In [38]:
# Check F1 score and Helstrom bound values for various rescale and n_copies values
model = HQC_original(rescale=0.5, n_copies=3).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted'), model.Hels_bound_

(0.7520661157024794, 0.877254248273404)

In [4]:
# Time required for n_copies=1
%timeit HQC_original(rescale=0.5, n_copies=1).fit(X_train, y_train)

4.5 ms ± 414 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
# Time required for n_copies=2
%timeit HQC_original(rescale=0.5, n_copies=2).fit(X_train, y_train)

30.4 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# Time required for n_copies=3
%timeit HQC_original(rescale=0.5, n_copies=3).fit(X_train, y_train)

3.68 s ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
# Time required for n_copies=4
%timeit HQC_original(rescale=0.5, n_copies=4).fit(X_train, y_train)

22min 20s ± 19.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# Time required for n_copies=5. Memory blow-out
%timeit HQC_original(rescale=0.5, n_copies=5).fit(X_train, y_train)

MemoryError: Unable to allocate 8.00 GiB for an array with shape (32768, 32768) and data type float64

In [29]:
# banana dataset (2 features, 5300 rows)
import pandas as pd

df = pd.read_csv('banana.tsv',delimiter='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

In [33]:
# Check F1 score and Helstrom bound values for various rescale and n_copies values
model = HQC_original(rescale=0.5, n_copies=4).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted'), model.Hels_bound_

(0.858978398722441, 0.7732939055876815)

In [20]:
# Time required for n_copies=1
%timeit HQC_original(rescale=0.5, n_copies=1).fit(X_train, y_train)

492 ms ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# Time required for n_copies=2
%timeit HQC_original(rescale=0.5, n_copies=2).fit(X_train, y_train)

804 ms ± 63.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# Time required for n_copies=3
%timeit HQC_original(rescale=0.5, n_copies=3).fit(X_train, y_train)

1.21 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# Time required for n_copies=4
%timeit HQC_original(rescale=0.5, n_copies=4).fit(X_train, y_train)

2.3 s ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# Time required for n_copies=5
%timeit HQC_original(rescale=0.5, n_copies=5).fit(X_train, y_train)

7.34 s ± 173 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# Time required for n_copies=6
%timeit HQC_original(rescale=0.5, n_copies=6).fit(X_train, y_train)

1min 26s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
# Time required for n_copies=7
%timeit HQC_original(rescale=0.5, n_copies=7).fit(X_train, y_train)

15min 36s ± 26.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
